# Asynchronous launches

Sometimes you may want to launch data labeling without blocking your interpreter – i.e. to continue doing some calculations and periodically checking labeling progress.

In this notebook, we consider asynchronous launch to implement multiple simultaneous data labeling processes. In general, it is sufficient to launch single data labeling process, but you may have some split of your data such that workers should see only objects from one split in their assignments. In code, we call such splits as _baskets_.

Parallel launches are implemented with Python's `multiprocessing` and `threading` modules. Interpreter spends most time on waiting for workers labeling by calling Toloka API methods, so it is mostly I/O bound task.

The code in this notebook overrides how the labeling is run, making separate `launch()` calls in individual threads. Also, since labeling process is not interactive (no prompts, metrics plotting and etc.), we just save labeling to files with basket names as a labeling results handling. You can handle results in your own way.

You can stop labeling process by interrupting the kernel as in usual synchronous launch, but please note that in asynchronous case it will take more time (up to several minutes) to handle interrupt since threads are needed to properly handle interrupt event to make graceful shutdown. Please do not repeat kernel interrupt while interpreter already handles previous one.

It is expected that you have already specified source data, labeling params and other variables necessary for labeling launch.

In [ ]:
from multiprocessing.pool import ThreadPool
from threading import Event

In [ ]:
def label_basket(basket_and_interrupted):
    basket_name, basket_input_objects, interrupted = basket_and_interrupted
    raw_results, worker_weights = client.launch(
        task_spec_en,
        params,
        basket_input_objects,
        control_objects,
        toloka_client,
        interactive=False,
        interrupted=interrupted,
    )
    results = client.ClassificationResults(basket_input_objects, raw_results, task_spec_en, worker_weights)
    results.predict().to_json(f'{basket_name}_results.json', orient='records')

In [ ]:
def launch_in_parallel(baskets):
    interrupted = Event()
    baskets_and_interrupted = [(name, objects, interrupted) for name, objects in baskets]
    tpool = ThreadPool(processes=len(baskets))
    try:
        tpool.map(label_basket, baskets_and_interrupted)
    except KeyboardInterrupt:
        interrupted.set()
        tpool.close()
        tpool.join()

In [ ]:
# your data split by baskets
baskets = [('red', input_objects[:100]), ('green', input_objects[100:200]), ('blue', input_objects[200:300])]

In [ ]:
launch_in_parallel(baskets)